<a href="https://colab.research.google.com/github/sr6awi/ieee_fraud_detection/blob/main/notebooks/04_modeling_dual_path_full_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("🚀 DUAL-PATH FRAUD DETECTION - COMPLETE TRAINING PIPELINE")
print("Node Classification with Full Dataset Support")
print("=" * 70)

# GPU Configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU: {len(gpus)} device(s) available")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU - using CPU (slower)")

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


🚀 DUAL-PATH FRAUD DETECTION - COMPLETE TRAINING PIPELINE
Node Classification with Full Dataset Support
⚠️ No GPU - using CPU (slower)


In [8]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 842.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 136.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 142.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00


In [10]:
BASE_DIR = Path("/content/drive/MyDrive/ML_Projects/ieee-fraud-detection")
PROCESSED_DIR = BASE_DIR / "processed"
MODEL_DIR = BASE_DIR / "models"
RESULTS_DIR = BASE_DIR / "results"

MODEL_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Training configuration
CONFIG = {
    'batch_size': 1024,
    'epochs': 50,
    'learning_rate': 0.001,
    'patience': 10,
    'n_folds': 5,
    'use_class_weights': True,
    'save_best_only': True
}

print(f"\n📁 Directories:")
print(f"   Data: {PROCESSED_DIR}")
print(f"   Models: {MODEL_DIR}")
print(f"   Results: {RESULTS_DIR}")


📁 Directories:
   Data: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed
   Models: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models
   Results: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/results


In [11]:
print("\n" + "="*70)
print("📥 LOADING FEATURE-ENGINEERED DATA")
print("="*70)

print("\nLoading training data...")
train = pd.read_csv(PROCESSED_DIR / "train_feature_engineered.csv")
print(f"✅ Train loaded: {train.shape}")

print("Loading test data...")
test = pd.read_csv(PROCESSED_DIR / "test_feature_engineered.csv")
print(f"✅ Test loaded: {test.shape}")

# Check for target
if 'isFraud' not in train.columns:
    raise ValueError("Target 'isFraud' not found in training data!")

print(f"\n📊 Data Statistics:")
print(f"   Training samples: {len(train):,}")
print(f"   Test samples: {len(test):,}")
print(f"   Features: {train.shape[1] - 2}")  # Exclude isFraud and TransactionID
print(f"   Fraud rate: {train['isFraud'].mean()*100:.2f}%")
print(f"   Class imbalance: {(1-train['isFraud'].mean())/train['isFraud'].mean():.1f}:1")


📥 LOADING FEATURE-ENGINEERED DATA

Loading training data...
✅ Train loaded: (590540, 262)
Loading test data...
✅ Test loaded: (506691, 261)

📊 Data Statistics:
   Training samples: 590,540
   Test samples: 506,691
   Features: 260
   Fraud rate: 3.50%
   Class imbalance: 27.6:1


In [12]:
print("\n" + "="*70)
print("🎯 PREPARING FEATURES FOR DUAL PATHS")
print("="*70)

# Smart features for GNN (features that benefit from graph structure)
GNN_FEATURES = [
    'TransactionAmt', 'TransactionDT',
    'card1', 'card2', 'card1_count',
    'card1_fraud_rate', 'card2_fraud_rate',
    'P_emaildomain_fraud_rate', 'R_emaildomain_fraud_rate',
    'addr1', 'dist1', 'ProductCD', 'ProductCD_fraud_rate',
    'transaction_hour', 'is_weekend', 'is_night',
    'TransactionAmt_log', 'email_domains_match', 'addr_match'
]

# Filter available GNN features
GNN_FEATURES = [f for f in GNN_FEATURES if f in train.columns]

print(f"✅ GNN Path: {len(GNN_FEATURES)} smart features selected")
print(f"   Focus: Graph-beneficial features")

# All features for TabNet (exclude ID and target)
exclude_cols = ['isFraud', 'TransactionID']
TABNET_FEATURES = [c for c in train.columns if c not in exclude_cols]

print(f"✅ TabNet Path: {len(TABNET_FEATURES)} features (all available)")
print(f"   Focus: Rich feature learning")


🎯 PREPARING FEATURES FOR DUAL PATHS
✅ GNN Path: 19 smart features selected
   Focus: Graph-beneficial features
✅ TabNet Path: 260 features (all available)
   Focus: Rich feature learning


In [13]:
print("\n" + "="*70)
print("📊 PREPARING TRAINING DATA (SIMPLIFIED FOR MEMORY)")
print("="*70)

"""
SIMPLIFIED APPROACH:
- Skip graph construction for now (memory intensive)
- Use GNN features as regular neural network
- Focus on TabNet path primarily
- Treat as multi-modal learning (two feature sets)
"""

# Prepare features
X_gnn = train[GNN_FEATURES].fillna(0).values
X_tabnet = train[TABNET_FEATURES].fillna(0).values
y = train['isFraud'].values

X_gnn_test = test[GNN_FEATURES].fillna(0).values
X_tabnet_test = test[TABNET_FEATURES].fillna(0).values

print(f"✅ Training data prepared:")
print(f"   GNN input shape: {X_gnn.shape}")
print(f"   TabNet input shape: {X_tabnet.shape}")
print(f"   Labels shape: {y.shape}")

# Normalize features
print("\n📏 Normalizing features...")
scaler_gnn = StandardScaler()
scaler_tabnet = StandardScaler()

X_gnn_scaled = scaler_gnn.fit_transform(X_gnn)
X_tabnet_scaled = scaler_tabnet.fit_transform(X_tabnet)

X_gnn_test_scaled = scaler_gnn.transform(X_gnn_test)
X_tabnet_test_scaled = scaler_tabnet.transform(X_tabnet_test)

# Save scalers
import joblib
joblib.dump(scaler_gnn, MODEL_DIR / 'scaler_gnn.pkl')
joblib.dump(scaler_tabnet, MODEL_DIR / 'scaler_tabnet.pkl')

print("✅ Features normalized and scalers saved!")


📊 PREPARING TRAINING DATA (SIMPLIFIED FOR MEMORY)
✅ Training data prepared:
   GNN input shape: (590540, 19)
   TabNet input shape: (590540, 260)
   Labels shape: (590540,)

📏 Normalizing features...
✅ Features normalized and scalers saved!


In [14]:
print("\n" + "="*70)
print("🏗️ BUILDING SIMPLIFIED DUAL-PATH MODEL")
print("="*70)

def build_dual_path_model(gnn_features, tabnet_features):
    """
    Simplified dual-path architecture for production training.
    Treats GNN features as a separate neural network path.
    """

    # INPUT LAYERS
    input_gnn = layers.Input(shape=(gnn_features,), name='gnn_input')
    input_tabnet = layers.Input(shape=(tabnet_features,), name='tabnet_input')

    # PATH 1: GNN-style Network (processes graph-beneficial features)
    x1 = layers.Dense(128, activation='relu', name='gnn_dense1')(input_gnn)
    x1 = layers.BatchNormalization(name='gnn_bn1')(x1)
    x1 = layers.Dropout(0.3, name='gnn_dropout1')(x1)

    x1 = layers.Dense(64, activation='relu', name='gnn_dense2')(x1)
    x1 = layers.BatchNormalization(name='gnn_bn2')(x1)
    x1 = layers.Dropout(0.2, name='gnn_dropout2')(x1)

    gnn_output = layers.Dense(32, activation='relu', name='gnn_output')(x1)

    # PATH 2: TabNet-style Network (attention-based feature selection)
    # Step 1
    x2 = layers.Dense(256, activation='gelu', name='tabnet_step1')(input_tabnet)
    x2 = layers.BatchNormalization(name='tabnet_bn1')(x2)
    x2 = layers.Dropout(0.3, name='tabnet_dropout1')(x2)

    # Step 2
    x2 = layers.Dense(128, activation='gelu', name='tabnet_step2')(x2)
    x2 = layers.BatchNormalization(name='tabnet_bn2')(x2)
    x2 = layers.Dropout(0.2, name='tabnet_dropout2')(x2)

    # Step 3
    tabnet_output = layers.Dense(32, activation='relu', name='tabnet_output')(x2)

    # FUSION: Concatenate both paths
    merged = layers.Concatenate(name='fusion_concat')([gnn_output, tabnet_output])

    # Attention-based fusion
    attention_weights = layers.Dense(64, activation='softmax', name='attention_weights')(merged)
    attended = layers.Multiply(name='attended_features')([merged, attention_weights])

    # Final classification layers
    x = layers.Dense(64, activation='relu', name='classifier_dense1')(attended)
    x = layers.BatchNormalization(name='classifier_bn')(x)
    x = layers.Dropout(0.2, name='classifier_dropout')(x)

    x = layers.Dense(32, activation='relu', name='classifier_dense2')(x)

    # Output
    output = layers.Dense(1, activation='sigmoid', name='output')(x)

    # Create model
    model = keras.Model(
        inputs=[input_gnn, input_tabnet],
        outputs=output,
        name='DualPathFraudDetector'
    )

    return model

# Build model
model = build_dual_path_model(len(GNN_FEATURES), len(TABNET_FEATURES))

print("✅ Model architecture built!")
print(f"\n📋 Model Summary:")
model.summary()


🏗️ BUILDING SIMPLIFIED DUAL-PATH MODEL
✅ Model architecture built!

📋 Model Summary:


Model: "DualPathFraudDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ gnn_input           │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_input        │ (None, 260)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_dense1 (Dense)  │ (None, 128)       │      2,560 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_step1        │ (None, 256)       │     66,816 │ tabnet_input[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_bn1             │ (None, 128)       │        512 │ gnn_dense1[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_bn1          │ (None, 256)       │      1,024 │ tabnet_step1[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_dropout1        │ (None, 128)       │          0 │ gnn_bn1[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_dropout1     │ (None, 256)       │          0 │ tabnet_bn1[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_dense2 (Dense)  │ (None, 64)        │      8,256 │ gnn_dropout1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_step2        │ (None, 128)       │     32,896 │ tabnet_dropout1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_bn2             │ (None, 64)        │        256 │ gnn_dense2[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_bn2          │ (None, 128)       │        512 │ tabnet_step2[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_dropout2        │ (None, 64)        │          0 │ gnn_bn2[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_dropout2     │ (None, 128)       │          0 │ tabnet_bn2[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_output (Dense)  │ (None, 32)        │      2,080 │ gnn_dropout2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabnet_output       │ (None, 32)        │      4,128 │ tabnet_dropout2[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_concat       │ (None, 64)        │          0 │ gnn_output[0][0], │
│ (Concatenate)       │                   │            │ tabnet_output[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_weights   │ (None, 64)        │      4,160 │ fusion_concat[0]

 Total params: 129,729 (506.75 KB)

 Trainable params: 128,449 (501.75 KB)

 Non-trainable params: 1,280 (5.00 KB)

In [15]:
print("\n" + "="*70)
print("⚙️ COMPILING MODEL")
print("="*70)

# Calculate class weights for imbalanced data
fraud_rate = y.mean()
class_weight = {
    0: 1.0,
    1: (1 - fraud_rate) / fraud_rate
}

print(f"📊 Class weights (for imbalance):")
print(f"   Class 0 (Legit): {class_weight[0]:.2f}")
print(f"   Class 1 (Fraud): {class_weight[1]:.2f}")

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.AUC(name='auc'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

print("\n✅ Model compiled successfully!")


⚙️ COMPILING MODEL
📊 Class weights (for imbalance):
   Class 0 (Legit): 1.00
   Class 1 (Fraud): 27.58

✅ Model compiled successfully!


In [5]:
!pip install keras-tqdm

In [16]:


print("\n" + "="*70)
print("🎓 TRAINING WITH STRATIFIED K-FOLD CROSS-VALIDATION")
print("="*70)

from tensorflow.keras.callbacks import LambdaCallback

# Base callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_auc',
        patience=CONFIG['patience'],
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_auc',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        mode='max',
        verbose=1
    )
]

# Stratified K-Fold setup
skf = StratifiedKFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=SEED)

cv_scores = []
fold_histories = []

print(f"\n🔄 Starting {CONFIG['n_folds']}-Fold Cross-Validation...")
print("="*70)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_gnn_scaled, y), 1):
    print(f"\n{'='*70}")
    print(f"📊 FOLD {fold}/{CONFIG['n_folds']}")
    print(f"{'='*70}")

    # Split data
    X_gnn_train, X_gnn_val = X_gnn_scaled[train_idx], X_gnn_scaled[val_idx]
    X_tabnet_train, X_tabnet_val = X_tabnet_scaled[train_idx], X_tabnet_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    print(f"   Train: {len(y_train):,} samples ({y_train.mean()*100:.2f}% fraud)")
    print(f"   Val:   {len(y_val):,} samples ({y_val.mean()*100:.2f}% fraud)")

    # Rebuild model (fresh weights for each fold)
    model = build_dual_path_model(len(GNN_FEATURES), len(TABNET_FEATURES))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.AUC(name='auc'),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall')
        ]
    )

    # Custom epoch logger for short summaries
    epoch_logger = LambdaCallback(
        on_epoch_end=lambda e, logs:
            print(f"Epoch {e+1:02d} | "
                  f"loss={logs['loss']:.4f} | val_loss={logs['val_loss']:.4f} | "
                  f"auc={logs['auc']:.4f} | val_auc={logs['val_auc']:.4f}")
    )

    # Create callbacks for this fold (unique checkpoint path)
    fold_callbacks = callbacks.copy() + [
        keras.callbacks.ModelCheckpoint(
            filepath=str(MODEL_DIR / f'best_model_fold_{fold}.keras'),
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        epoch_logger
    ]

    # Train
    print(f"\n   🚀 Training...")
    history = model.fit(
        [X_gnn_train, X_tabnet_train],
        y_train,
        validation_data=([X_gnn_val, X_tabnet_val], y_val),
        epochs=CONFIG['epochs'],
        batch_size=CONFIG['batch_size'],
        class_weight=class_weight if CONFIG['use_class_weights'] else None,
        callbacks=fold_callbacks,
        verbose=1      # ✅ live progress bar
    )

    # Evaluate
    val_preds = model.predict([X_gnn_val, X_tabnet_val], verbose=0)
    val_auc = roc_auc_score(y_val, val_preds)
    cv_scores.append(val_auc)
    fold_histories.append(history.history)

    print(f"\n   ✅ Fold {fold} Complete!")
    print(f"   Val AUC: {val_auc:.6f}")
    print(f"   Best epoch: {np.argmax(history.history['val_auc']) + 1}")

# ============================================================================
# RESULTS SUMMARY
# ============================================================================

print("\n" + "="*70)
print("📊 CROSS-VALIDATION RESULTS")
print("="*70)

print(f"\nFold AUC Scores:")
for i, score in enumerate(cv_scores, 1):
    print(f"   Fold {i}: {score:.6f}")

print(f"\n🏆 Summary:")
print(f"   Mean AUC: {np.mean(cv_scores):.6f}")
print(f"   Std AUC:  {np.std(cv_scores):.6f}")
print(f"   Min AUC:  {np.min(cv_scores):.6f}")
print(f"   Max AUC:  {np.max(cv_scores):.6f}")

# Save cross-validation results
cv_results = {
    'fold_scores': cv_scores,
    'mean_auc': float(np.mean(cv_scores)),
    'std_auc': float(np.std(cv_scores)),
    'config': CONFIG
}

with open(RESULTS_DIR / 'cv_results.json', 'w') as f:
    json.dump(cv_results, f, indent=2)

print(f"\n✅ CV results saved to: {RESULTS_DIR / 'cv_results.json'}")


🎓 TRAINING WITH STRATIFIED K-FOLD CROSS-VALIDATION

🔄 Starting 5-Fold Cross-Validation...

📊 FOLD 1/5
   Train: 472,432 samples (3.50% fraud)
   Val:   118,108 samples (3.50% fraud)

   🚀 Training...
Epoch 1/50
458/462 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7618 - auc: 0.8389 - loss: 0.9502 - precision: 0.1100 - recall: 0.7568
Epoch 1: val_auc improved from -inf to 0.91710, saving model to /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/best_model_fold_1.keras
Epoch 01 | loss=0.8252 | val_loss=0.2830 | auc=0.8832 | val_auc=0.9171
462/462 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.7624 - auc: 0.8394 - loss: 0.9488 - precision: 0.1103 - recall: 0.7571 - val_accuracy: 0.9227 - val_auc: 0.9171 - val_loss: 0.2830 - val_precision: 0.2721 - val_recall: 0.7217 - learning_rate: 0.0010
Epoch 2/50
457/462 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8475 - auc: 0.9123 - loss: 0.7238 - precision: 0.1648 - recall: 0.8122
Epoch 2: val_auc improved from 0.91710 to 0.

In [17]:
print("\n" + "="*70)
print("🎯 TRAINING FINAL MODEL ON FULL DATASET")
print("="*70)

# Rebuild model
final_model = build_dual_path_model(len(GNN_FEATURES), len(TABNET_FEATURES))
final_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc'),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall')]
)

# Use 80/20 split for final training
from sklearn.model_selection import train_test_split

X_gnn_train, X_gnn_val, X_tabnet_train, X_tabnet_val, y_train_final, y_val_final = train_test_split(
    X_gnn_scaled, X_tabnet_scaled, y,
    test_size=0.2,
    stratify=y,
    random_state=SEED
)

print(f"   Training samples: {len(y_train_final):,}")
print(f"   Validation samples: {len(y_val_final):,}")

# Callbacks for final model
final_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_auc',
        patience=CONFIG['patience'],
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_auc',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        mode='max',
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=str(MODEL_DIR / 'final_model_best.h5'),
        monitor='val_auc',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    keras.callbacks.CSVLogger(
        filename=str(RESULTS_DIR / 'training_log.csv')
    )
]

print("\n🎓 Training final model...")
final_history = final_model.fit(
    [X_gnn_train, X_tabnet_train],
    y_train_final,
    validation_data=([X_gnn_val, X_tabnet_val], y_val_final),
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    class_weight=class_weight if CONFIG['use_class_weights'] else None,
    callbacks=final_callbacks,
    verbose=1
)

print("\n✅ Final model training complete!")

# Save final model
final_model.save(MODEL_DIR / 'final_model.h5')
print(f"✅ Final model saved: {MODEL_DIR / 'final_model.h5'}")


🎯 TRAINING FINAL MODEL ON FULL DATASET
   Training samples: 472,432
   Validation samples: 118,108

🎓 Training final model...
Epoch 1/50
457/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7557 - auc: 0.8265 - loss: 0.9733 - precision: 0.1054 - recall: 0.7418
Epoch 1: val_auc improved from -inf to 0.92227, saving model to /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/final_model_best.h5


462/462 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.7564 - auc: 0.8271 - loss: 0.9716 - precision: 0.1057 - recall: 0.7423 - val_accuracy: 0.9084 - val_auc: 0.9223 - val_loss: 0.3107 - val_precision: 0.2416 - val_recall: 0.7566 - learning_rate: 0.0010
Epoch 2/50
457/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8531 - auc: 0.9144 - loss: 0.7112 - precision: 0.1678 - recall: 0.8107
Epoch 2: val_auc improved from 0.92227 to 0.93063, saving model to /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/final_model_best.h5


462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8531 - auc: 0.9144 - loss: 0.7111 - precision: 0.1678 - recall: 0.8108 - val_accuracy: 0.8579 - val_auc: 0.9306 - val_loss: 0.3459 - val_precision: 0.1772 - val_recall: 0.8396 - learning_rate: 0.0010
Epoch 3/50
458/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8658 - auc: 0.9259 - loss: 0.6630 - precision: 0.1831 - recall: 0.8214
Epoch 3: val_auc improved from 0.93063 to 0.93603, saving model to /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/final_model_best.h5


462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8658 - auc: 0.9259 - loss: 0.6630 - precision: 0.1830 - recall: 0.8215 - val_accuracy: 0.8622 - val_auc: 0.9360 - val_loss: 0.3182 - val_precision: 0.1835 - val_recall: 0.8514 - learning_rate: 0.0010
Epoch 4/50
461/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8679 - auc: 0.9329 - loss: 0.6322 - precision: 0.1881 - recall: 0.8396
Epoch 4: val_auc improved from 0.93603 to 0.93913, saving model to /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/final_model_best.h5


462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.8679 - auc: 0.9329 - loss: 0.6322 - precision: 0.1881 - recall: 0.8396 - val_accuracy: 0.8720 - val_auc: 0.9391 - val_loss: 0.3047 - val_precision: 0.1952 - val_recall: 0.8505 - learning_rate: 0.0010
Epoch 5/50
462/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8761 - auc: 0.9376 - loss: 0.6108 - precision: 0.1986 - recall: 0.8402
Epoch 5: val_auc improved from 0.93913 to 0.94130, saving model to /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/final_model_best.h5


...

462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8971 - auc: 0.9639 - loss: 0.4684 - precision: 0.2388 - recall: 0.8912 - val_accuracy: 0.9325 - val_auc: 0.9516 - val_loss: 0.1770 - val_precision: 0.3188 - val_recall: 0.8161 - learning_rate: 0.0010
Epoch 18/50
460/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9003 - auc: 0.9660 - loss: 0.4535 - precision: 0.2456 - recall: 0.8954
Epoch 18: val_auc improved from 0.95161 to 0.95275, saving model to /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/final_model_best.h5


462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.9003 - auc: 0.9660 - loss: 0.4535 - precision: 0.2456 - recall: 0.8955 - val_accuracy: 0.9353 - val_auc: 0.9527 - val_loss: 0.1693 - val_precision: 0.3270 - val_recall: 0.8021 - learning_rate: 0.0010
Epoch 19/50
457/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9009 - auc: 0.9675 - loss: 0.4446 - precision: 0.2478 - recall: 0.9030
Epoch 19: val_auc did not improve from 0.95275
462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9009 - auc: 0.9675 - loss: 0.4447 - precision: 0.2477 - recall: 0.9030 - val_accuracy: 0.9359 - val_auc: 0.9502 - val_loss: 0.1590 - val_precision: 0.3299 - val_recall: 0.8074 - learning_rate: 0.0010
Epoch 20/50
461/462 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8977 - auc: 0.9678 - loss: 0.4419 - precision: 0.2420 - recall: 0.9049
Epoch 20: val_auc did not improve from 0.95275
462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8977 - auc: 0.9678 - loss: 0.4419 - precision: 0.2420 - reca

462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8991 - auc: 0.9701 - loss: 0.4257 - precision: 0.2455 - recall: 0.9112 - val_accuracy: 0.9340 - val_auc: 0.9543 - val_loss: 0.1658 - val_precision: 0.3243 - val_recall: 0.8176 - learning_rate: 0.0010
Epoch 23/50
460/462 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9041 - auc: 0.9704 - loss: 0.4233 - precision: 0.2545 - recall: 0.9069
Epoch 23: val_auc did not improve from 0.95435
462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9041 - auc: 0.9704 - loss: 0.4233 - precision: 0.2545 - recall: 0.9069 - val_accuracy: 0.9424 - val_auc: 0.9507 - val_loss: 0.1468 - val_precision: 0.3549 - val_recall: 0.7895 - learning_rate: 0.0010
Epoch 24/50
459/462 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9054 - auc: 0.9705 - loss: 0.4218 - precision: 0.2573 - recall: 0.9062
Epoch 24: val_auc did not improve from 0.95435
462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9054 - auc: 0.9705 - loss: 0.4217 - precision: 0.2573 - reca

462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.9057 - auc: 0.9725 - loss: 0.4077 - precision: 0.2583 - recall: 0.9103 - val_accuracy: 0.9460 - val_auc: 0.9546 - val_loss: 0.1455 - val_precision: 0.3731 - val_recall: 0.7997 - learning_rate: 0.0010
Epoch 27/50
457/462 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9115 - auc: 0.9739 - loss: 0.3974 - precision: 0.2716 - recall: 0.9139
Epoch 27: val_auc did not improve from 0.95455
462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9114 - auc: 0.9739 - loss: 0.3975 - precision: 0.2715 - recall: 0.9139 - val_accuracy: 0.9436 - val_auc: 0.9523 - val_loss: 0.1475 - val_precision: 0.3611 - val_recall: 0.7958 - learning_rate: 0.0010
Epoch 28/50
459/462 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9086 - auc: 0.9734 - loss: 0.4007 - precision: 0.2656 - recall: 0.9166
Epoch 28: val_auc did not improve from 0.95455
462/462 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9086 - auc: 0.9734 - loss: 0.4007 - precision: 0.2655 - reca


✅ Final model training complete!
✅ Final model saved: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/models/final_model.h5
